# 1 Overview of Speech Recognition
## 1.1 Hidden Markov Model for Accoustic Modeling
As speech is 


# 2 Hidden Markov Model for Accoustic Modeling
## Phoneme model of words


# 2.1 Gaussian Mixture Model
## Overview


+ **Mixture of Gaussians**
$$p(x) = \sum_{k=1}^{K} \pi_{k}N(x|\mu_{k}, \Sigma_{k})$$
> + Add a hidden variable Z, which is like a swith to each Gaussian
  $$p(Z) = \prod_{k=1}^{K} \pi_{k}^{z_{k}}$$
  + each Gaussian can be expressed as x depends on $z_{k}$
  $$p(x | z_{k} = 1) = N(x|\mu_{k}, \Sigma_{k})$$
  and
  $$p(x | Z) = \prod_{k=1}^{K} N(x|\mu_{k}, \Sigma_{k})^{z_{k}}$$
  + Marginal Distribution of x
  $$p(x) = \sum_{z} p(x|z) p(z) = \sum_{k=1}^{K} \pi_{k} N(x|\mu_{k}, \Sigma_{k})$$
  + Responsity of a Gaussion
  $$\begin{aligned}\gamma(z_{k}) \equiv p(z_{k}=1|x) = \frac{p(z_{k}=1)p(x|z_{k}=1)}{\sum_{j=1}^{K} p(z_{j}=1)p(x|z_{j}=1)} 
     = \frac{\pi_{k}N(x|\mu_{k}, \Sigma_{k})}{\sum_{j=1}^{K} \pi_{j}N(x|\mu_{j}, \Sigma_{j})} \end{aligned}$$
  

+ **EM**
> + likelihood function
    $$ p(X|\pi, \mu, \Sigma) = \prod_{n=1}^{N} p(x_{n}|\pi,\mu,\Sigma) 
    = \prod_{n=1}^{N}\sum_{k=1}^{K}\pi_{k}N(x_{n}|\mu_{k}, \Sigma_{k}) $$
    Take log and get log-likelihood
    $$\ln p(X|\pi, \mu, \Sigma) = \sum_{n=1}^{N} \ln \bigg\{ \sum_{k=1}^{K}\pi_{k}N(x_{n}|\mu_{k}, \Sigma_{k}) \bigg\} $$
  + set Derivative of the log-likelihood w.r.t $\pi$, $\mu$, $\Sigma$ to ***Zero*** <br/>
    Since there is a constraint with $\pi$: $\sum_{\pi_{k}}^{K} = 1$, we should use ***Langrange multiplier*** method.
    >> + Derivative of log-likelihood w.r.t $\mu_{k}$
    $$N(x|\mu, \Sigma) = \frac{1}{(2\pi)^\frac{D}{2}|\Sigma|^{\frac{1}{2}}} 
    exp\bigg\{ -\frac{1}{2}(x-\mu)^{T}\Sigma^{-1}(x-\mu)  \bigg\}$$
    $$\nabla_{\mu_{k}} \ln p(X|\pi, \mu, \Sigma) = 
    \sum_{n=1}^{N} \frac{\pi_{k}N(x_{n}|\mu_{k}, \Sigma_{k})}{\Sigma_{j=1}^{K} \pi_{j}N(x_{n}|\mu_{j}, \Sigma_{n})}
    \Sigma_{k}^{-1} (x_{n} - \mu_{k}) = 0 $$    
    $$\mu_{k} = \frac{1}{N_{k}} \sum_{n=1}^{N} \gamma (z_{nk}) x_{n}$$
    where $N_{k} = \sum_{n=1}{N} \gamma (z_{nk}) $
    + Derivative w.r.t $\Sigma_{k}$   
    $$\Sigma_{k} = \frac{1}{N_{k}} \sum_{n=1}^{N} \gamma(z_{nk}) (x_n-\mu_k)(x_n-\mu_k)^T$$

    
    
         
    
  

+ **VEM**
> + Variational Method

+ **Choose of Covariance** 
> + Diagonal
> + Full



In [1]:
"""This implementation only focus on diagonal Gaussians
"""

import numpy as np
from numpy import linalg as la
class GMM:
    def __init__(self):
        EM_init()
    
    def Gauss(x, mu, Sigma):
        """Multivariate diagnal covariance Gaussian"""
        return np.exp(-np.dot((x-mu)/Sigma, x-mu)) / (np.power(2 * np.pi, len(mu)/ 2) * np.sqrt(np.prod(Sigma)))

    def log_likelihood(self, X):
        """Compute log likelihood"""
        Mus, Pis, Sigmas = (self.Mus, self.Pis, self.Sigmas)
        assert(len(Mus) == len(Pis)==len(Simgas))
        K= len(Mus)
        ll = np.sum(np.log(SumGauss(x, Mus, Pis, Sigmas)) for x in X)
        return ll
        
        
    def SumGauss(x, Mus, Sigmas, Pis):
        return np.sum([Pis[i] * Gauss(x, Mus[i],Sigmas[i]) for i in range(len(Mus))])
    
    def EM_init(self, Mus, Sigmas, Pis):
        """Initialize EM"""
        
    def EM(X, Mus_init, Sigmas_init, Pis_init):
        """Expectation Maximization Algorithm""" 
        N = len(X)
        # E-step
        Mus, Sigmas, Pis = (Mus_init, Sigmas_init, Pis_init)
        K = len(Mus)
        def E(x, Mus, Sigmas, Pis):
            g = SumGauss(x, Mus, Sigmas, Pis)
            s = np.sum(g)
            return g/s
        
        rk = np.array([E(x, Mus, Sigmas, Pis) for x in X])
        Nk = np.sum(rk, axis=0)
    
        # M-step
        Pis_new = Nk/N
        Mus_new = np.array([np.sum([rk[n,k] * X[n] for n in range(0,N)], axis = 0) for k in range(0, K)])
        def diag_M():
            
            
            
     
        
    
    
    
    
    def VEM():
        """Variational EM"""
    
    def GMM(data, K, covtype="diagonal"):
        """Gaussian Mixture Model using Variational EM"""
    


In [24]:
k = 3
d = 2
Pis = np.array([.3,.3])
Mus = np.array([[0.2, 0.7, 0.9], [2, 3, 4]])
Sigmas = np.array([[0.4,.6, 0.7],[0.4,.3, 0.7]])

def Gauss(x, mu, Sigma):
    """Multivariate diagnal covariance Gaussian"""
    return np.exp(-np.dot((x-mu)/Sigma, x-mu)) / (np.power(2 * np.pi, len(mu)/ 2) * np.sqrt(np.prod(Sigma)))

X=np.array([[1,2,3],[4,5,6],[7,9,8]])
#g = [Pis[i] * Gauss(x, Mus[i],Sigmas[i]) for i in range(0, len(Mus))]
#g1 = Gauss(x.reshape((3,1)), Mus[1].reshape((3,1)),Sigmas[1])
#print("g1: ",g1)


def E(x, Mus, Sigmas, Pis):
        g = np.array([Pis[i] * Gauss(x, Mus[i],Sigmas[i]) for i in range(len(Mus))])
        s = np.sum(g)
        return g/s
    


    


rk = np.array([E(x, Mus, Sigmas, Pis) for x in X])
nk = np.sum(rk, axis=0)
nk



array([ 0.02185271,  2.97814729])

In [15]:
np.array([Pis[i] * Gauss(x, Mus[i],Sigmas[i]) for i in range(len(Mus))])


NameError: name 'x' is not defined

## 2.2 GMM in sklearn

In [2]:
import numpy as np
from scipy import linalg
from sklearn import mixture
n_comp = 64
gmm = mixture.GMM(n_components=n_comp, covariance_type='diag')




In [28]:
(np.array([1,2,3]) / (np.array([1,2,3]))).dot(np.array([1,2,3]))
np.prod(np.array([1,2,3]))
np.dot(np.array([1,2,3]).T, np.array([1,2,3]))

14

# 3 Hidden Markov Model